In [1]:
import networkx as nx
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import math
import scipy
import matplotlib.pyplot as plt 
import random

In [89]:
bodyData = pd.read_csv("soc-redditHyperlinks-body.tsv", sep='\t')
titleData = pd.read_csv("soc-redditHyperlinks-title.tsv", sep='\t')
df = bodyData.append(titleData, ignore_index=True)
del df['PROPERTIES']

In [3]:
grouped_df =  df[['SOURCE_SUBREDDIT','TARGET_SUBREDDIT','LINK_SENTIMENT']]
df2 = grouped_df.groupby(
     ['SOURCE_SUBREDDIT','TARGET_SUBREDDIT']
 ).agg(
     SENTIMENT_SUM = ('LINK_SENTIMENT','sum')
 ).reset_index()

In [4]:
df2['SENTIMENT_SUM_ABS'] = abs(df2['SENTIMENT_SUM'])
df2 = df2.sort_values('SENTIMENT_SUM_ABS',ascending=False).reset_index(drop=True)

In [32]:
negative_sum = df2.loc[df2['SENTIMENT_SUM']<0]

In [80]:
top_100_nodes = []
top_100_nodes_negative = []
top_100_nodes_positive_source = []
top_100_nodes_positive_target = []
top_100_nodes_negative_source = []
top_100_nodes_negative_target = []

In [81]:
df2.head()

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,SENTIMENT_SUM,SENTIMENT_SUM_ABS
0,trendingsubreddits,changelog,548,548
1,moronicmondayandroid,android,478,478
2,goodshibe,dogecoin,299,299
3,nightlypick,hockey,296,296
4,titlegore,todayilearned,279,279


In [82]:
top_100_nodes = []
for idx, row in df2.iterrows():
    
    top_100_nodes.append(row['SOURCE_SUBREDDIT'])
    top_100_nodes.append(row['TARGET_SUBREDDIT'])
    if len(set(top_100_nodes)) == 100:
        top_100_nodes = list(dict.fromkeys(top_100_nodes))
        print(idx)
        break

75


In [83]:
top_100_nodes_negative = []
for idx, row in negative_sum.iterrows():
    top_100_nodes_negative.append(row['SOURCE_SUBREDDIT'])
    top_100_nodes_negative.append(row['TARGET_SUBREDDIT'])
    if len(set(top_100_nodes_negative)) == 100:
        top_100_nodes_negative = list(dict.fromkeys(top_100_nodes_negative))
        print(idx)
        break

26270


In [84]:
top_100_nodes_positive_source = df2['SOURCE_SUBREDDIT'][:157].to_list()
top_100_nodes_positive_source = list(dict.fromkeys(top_100_nodes_positive_source))

In [85]:
top_100_nodes_positive_target = df2['TARGET_SUBREDDIT'][:169].to_list()
top_100_nodes_positive_target = list(dict.fromkeys(top_100_nodes_positive_target))

In [86]:
top_100_nodes_negative_target = negative_sum['TARGET_SUBREDDIT'][:127].to_list()
top_100_nodes_negative_target = list(dict.fromkeys(top_100_nodes_negative_target))

In [88]:
top_100_nodes_negative_source = negative_sum['SOURCE_SUBREDDIT'][:218].to_list()
top_100_nodes_negative_source = list(dict.fromkeys(top_100_nodes_negative_source))

In [103]:
Graph_n = nx.from_pandas_edgelist(df,source='SOURCE_SUBREDDIT', target='TARGET_SUBREDDIT', edge_attr=['LINK_SENTIMENT'], create_using=nx.DiGraph())

In [104]:
def create_subgraph(Graph_n,rankingtop100):  
    connected_nodes1 = []
    for u in range(len(rankingtop100)):
        connected_nodes1.append(list(Graph_n.neighbors(rankingtop100[u])))

    amount_of_connected_nodes = np.zeros(len(rankingtop100))

    connected_nodes1_random = []

    for u in range(len(rankingtop100)):
        amount_of_connected_nodes = int(0.2 * len(connected_nodes1[u]))
        connected_nodes1_random.append(random.sample(connected_nodes1[u], amount_of_connected_nodes))

    temp_list = []

    for u in range(len(connected_nodes1_random)):
        for i in connected_nodes1_random[u]:
            if i not in temp_list:
                temp_list.append(i)

    unique_connected_nodes = []
    for u in range(len(connected_nodes1_random)):
        unique_connected_nodes.append(set(connected_nodes1_random[u]) & set(temp_list))

    connected_nodes2 = []
    for u in range(len(rankingtop100)):
        for i in range(len(unique_connected_nodes[u])):
            connected_nodes2.append(list(Graph_n.neighbors(list(unique_connected_nodes[u])[i])))

    connected_nodes2_random = []
    connected_nodes2_without_nodes1 = []

    for u in range(len(rankingtop100)):
        connected_nodes2_without_nodes1 = list(set(connected_nodes2[u])^set(connected_nodes1[u]))
        amount_of_connected_nodes = int(0.2 * len(connected_nodes2_without_nodes1))
        connected_nodes2_random.append(random.sample(connected_nodes2_without_nodes1, amount_of_connected_nodes))


    final_node_set = []

    for u in range(len(rankingtop100)):
        final_neighbor_set = set(connected_nodes2_random[u]) | set(connected_nodes1_random[u])
        final_node_set.append(final_neighbor_set | set([rankingtop100[u]]))

    final_node_list = []
    for u in range(len(final_node_set)):
        final_node_list.append(list(final_node_set[u]))


    final_node_list_unique = []

    for u in range(len(final_node_set)):
        for i in final_node_list[u]:
            if i not in final_node_list_unique:
                final_node_list_unique.append(i)


    ranked_graph = Graph_n.subgraph(list(final_node_list_unique))
    
    return ranked_graph

In [112]:
G = create_subgraph(Graph_n,top_100_nodes)

In [113]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 4672
Number of edges: 122912
Average in degree:  26.3082
Average out degree:  26.3082


In [114]:
nx.number_strongly_connected_components(G)

587

In [115]:
nx.number_weakly_connected_components(G)

1